In [151]:
import pandas as pd
import numpy as np
import seaborn as sns
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN
import warnings
warnings.filterwarnings("ignore")

In [152]:
returns = pd.read_pickle("../Data/returns.pkl")
returns

,905270,921795,904261,905261,916328,923024,936365,902355,912215,929813,...,9660J1,69568X,543755,77463M,29235J,131745,69487D,68157P,9110RA,292703
date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,-0.835165,-0.475059,5.000000,-0.810537,0.361421,0.000000,-3.030303,0.000000,-0.431034,-1.315789,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-04,-0.797872,-1.272872,2.380952,-1.634321,-0.180060,3.454545,-1.562500,0.530612,0.225108,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,-2.546917,1.531023,-0.372093,-1.661475,-1.079730,1.054482,-3.174603,0.487211,-0.656531,-1.333333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,2.017423,-0.873016,0.000000,2.534319,0.000000,0.000000,0.000000,1.010101,0.434783,3.405405,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-24,NaN,NaN,NaN,NaN,0.136600,NaN,0.349877,NaN,NaN,NaN,...,0.376702,4.400221,1.590198,-0.991004,1.682900,3.364703,2.271817,0.634962,4.859794,1.204016
2021-05-25,NaN,NaN,NaN,NaN,0.946372,NaN,0.542355,NaN,NaN,NaN,...,0.230947,-0.288569,0.449063,-1.661106,0.439588,-0.548765,0.185928,0.727530,1.884279,-0.498203
2021-05-26,NaN,NaN,NaN,NaN,-1.393581,NaN,0.616491,NaN,NaN,NaN,...,1.296083,2.388001,-0.166049,3.255613,2.317760,4.702784,0.673960,2.336242,-0.429666,-1.520393


In [153]:
returns = returns.iloc[1:]

In [154]:
# dropping columns with all NaN
drop_columns = []

for col in returns.columns:
    if returns[col].isnull().all() == True:
        drop_columns.append(col)
        
returns.drop(columns=drop_columns, inplace=True)

In [155]:
def get_investable(t, n_rows):
    "Find stocks in investable universe at time t\
    (stocks in the S&P500 that have prices recorded for the last n_rows days)"
    
    df_investable = returns.copy(deep = True).sort_index(ascending = False)
    
    most_recent = t
    least_recent = t + pd.DateOffset(-n_rows)
    
    #take n_rows worth of data upto time specified
    df_investable = df_investable[most_recent+pd.DateOffset(-1):least_recent]
    
    #find all stocks that exist in the S&P at this time period
    investable_universe = []
    for col in df_investable.columns:
        if ~df_investable[col].iloc[:n_rows].isna().any():
            investable_universe.append(col)
        
    df_investable = df_investable[investable_universe]
    
    return df_investable

In [156]:
t = pd.to_datetime("2020-02-25")

get_investable(t, 80)

,916328,936365,905271,905113,905802,905425,906156,916305,992816,921093,...,311917,69568X,543755,77463M,29235J,131745,69487D,68157P,9110RA,292703
date,,,,,,,,,,,,,,,,,,,,,
2020-02-24,-3.224700,-7.807808,-3.702396,-1.195098,-6.519714,-1.641923,-4.974055,-5.967078,-4.750040,-2.460137,...,-2.020070,-7.459489,-2.467504,-6.481638,-6.091830,-4.945782,-4.950192,-2.934848,-7.530828,-3.530293
2020-02-21,-1.141759,-6.966998,-0.168363,-0.682511,-0.463535,-0.654425,-1.230048,-2.409639,-2.263503,-0.746100,...,-1.364204,0.176838,-1.604162,0.595846,-4.036727,-0.943149,-3.189911,-0.666838,-1.943285,-1.020350
2020-02-20,-1.007162,-2.767402,-0.164178,-1.085978,1.809029,-1.082126,-0.255605,0.789314,-1.025895,0.591312,...,-0.471153,-1.963115,0.348053,2.656414,-1.813720,2.884097,-0.111152,0.610700,0.028798,-1.631715
2020-02-19,0.540054,3.533134,0.031282,2.682395,3.974485,-0.795553,0.980826,0.816160,1.448276,-1.079865,...,-1.072034,6.875586,0.767207,42.444610,1.149301,0.487541,0.806753,4.644464,0.915432,1.375625
2020-02-18,-0.869953,2.856626,-0.105465,-0.539684,1.325820,0.549345,-0.198719,1.093234,-1.831051,-0.714764,...,1.049026,7.295914,0.573192,-2.238988,-3.065014,0.135612,-2.341698,-1.907116,-0.159582,-0.317316
2020-02-14,1.105097,1.430405,0.443346,1.244447,0.583431,0.884784,1.048639,-4.810524,0.024625,-1.104484,...,2.371351,-0.493781,1.024499,-0.604741,-0.125372,-3.683386,-1.160862,-2.151188,1.352742,2.426001
2020-02-13,-0.123888,1.187604,0.306966,0.426659,2.075889,1.041360,0.915641,-6.171702,-0.712103,-0.745451,...,0.422572,4.784371,1.790977,-0.385542,0.367011,2.737520,0.623957,1.785086,2.409276,0.566159
2020-02-12,-0.426152,0.167286,0.669546,2.467058,4.647436,-0.420661,0.459926,-0.348816,2.374769,-1.105811,...,-0.920232,-0.915507,0.730760,4.639435,0.638455,1.236245,1.704545,3.166107,1.389313,-1.148165
2020-02-11,0.985277,2.946804,0.278098,1.169012,2.202002,0.294349,0.294918,0.609531,-0.603328,0.963222,...,0.764667,0.401864,-0.091262,2.719503,2.893751,1.363261,1.749381,0.850869,2.088529,9.877013


In [157]:
# Converts investable dataframe into 3D tensor for input into RNN fitting
# Eg. if investable_df has shape (52, 635), it is converted to a numpy array of shape (52, 1, 635)

def formatX(investable_df):
    train_values = []
    # Iterates through each day in investable df and appends feature values to train_values
    for i in range(len(investable_df.index)):
        train_values.append(investable_df.iloc[i].values)
    train_values = np.array(train_values) # converts to numpy array
    train_values = np.reshape(train_values, (train_values.shape[0], 1, train_values.shape[1])) # reshapes to 3-dimensional
    return train_values

In [158]:
formatX(get_investable(t, 80))

array([[[-3.22469983e+00, -7.80780781e+00, -3.70239636e+00, ...,
         -2.93484809e+00, -7.53082795e+00, -3.53029256e+00]],

       [[-1.14175899e+00, -6.96699843e+00, -1.68363352e-01, ...,
         -6.66837651e-01, -1.94328487e+00, -1.02035000e+00]],

       [[-1.00716204e+00, -2.76740238e+00, -1.64177938e-01, ...,
          6.10700155e-01,  2.87976962e-02, -1.63171470e+00]],

       ...,

       [[ 4.13907285e-01,  7.60649087e-02,  0.00000000e+00, ...,
         -9.65078102e-01,  1.70616114e-01,  6.83854202e-03]],

       [[ 8.22701800e-01,  1.31004367e+00, -5.71874188e-01, ...,
          1.46375934e+00, -3.21239607e-01,  1.05037662e+00]],

       [[-1.88348152e+00, -1.76633863e+00, -5.21484291e-01, ...,
         -2.63416552e+00, -1.18569695e+00, -1.03945839e+00]]])

In [159]:
# Formats output for a stock as a np array
# Takes in a stock, a date, and a number of rows to look back on and formats the output values as a numpy array
# Eg. formatY(stockX, Feb.25, 80) outputs the returns for stockX starting from 80 days before Feb.25 up to Feb.25

def formatY(stock, t, n_rows):
    most_recent = t
    least_recent = t + pd.DateOffset(-n_rows)
    return returns[stock].loc[least_recent:most_recent].shift(-1).iloc[:-1].values[::-1]

In [160]:
# Creating a Simple RNN for an inputted stock

def get_model(stock, t, n_rows, investable_df):
    # Converts features and output values to a 3D tensor and numpy array respectively
    convertedX = convertToMatrix(investable_df)
    convertedY = formatY(stock, t, n_rows)
    
    # Builds model with three layers (SimpleRNN, Dense, Dense)
    model = Sequential()
    model.add(SimpleRNN(units=32, input_shape=(convertedX.shape[1], convertedX.shape[2]), activation="relu"))
    model.add(Dense(8, activation="relu")) 
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='rmsprop')
    
    # Fits model on formatted inputs and outputs
    model.fit(convertedX, convertedY, epochs=100, batch_size=16, verbose=0)
    
    return model

In [161]:
# Predicts stock price at a given date
import datetime

# Number of days to look back at when training model
look_back = 80

def pred_date(stock, date):
    investable = get_investable(date, look_back)
    
    # Checking for invalid inputs
    if stock not in investable.columns:
        print(stock + " is not in the investable universe")
        return
    if investable.shape[0] == 0:
        print("Investable universe is empty")
        return
    
    ''' investable.iloc[0:1] is the first row of the investable df 
    (i.e. the row we're using to predict the stock's return at time date) '''
    convertedX = formatX(investable.iloc[0:1])
    
    ''' We train the model on the day before date (the parameter) so that the date we're predicting
    for is not included in the training set'''
    model = get_model(stock, date - datetime.timedelta(days=1), look_back, investable)
    
    return model.predict(convertedX)[0, 0]

In [162]:
pred_date("916328", t)

-5.5258465

In [163]:
# Predicts returns for all stocks in the investable universe at a given date
def pred_date_all(date):
    investable = get_investable(date, look_back)
    df = pd.DataFrame()
    stocks = []
    predictions = []
    count = 0 # used to keep track of progress when running function
    for stock in investable.columns:
        stocks.append(stock)
        predictions.append(pred_date(stock, date))
        count += 1
        percent = count / len(investable.columns)
        print(str(percent * 100) + "% ")
    df["stock"] = stocks
    df["pred_" + date.strftime("%Y-%m-%d")] = predictions
    return df

In [164]:
pred_t = pred_date_all(t)

0.15748031496062992% 
0.31496062992125984% 
0.47244094488188976% 
0.6299212598425197% 
0.7874015748031495% 
0.9448818897637795% 
1.1023622047244095% 
1.2598425196850394% 
1.4173228346456692% 
1.574803149606299% 
1.7322834645669292% 
1.889763779527559% 
2.047244094488189% 
2.204724409448819% 
2.3622047244094486% 
2.5196850393700787% 
2.677165354330709% 
2.8346456692913384% 
2.9921259842519685% 
3.149606299212598% 
3.3070866141732282% 
3.4645669291338583% 
3.622047244094488% 
3.779527559055118% 
3.937007874015748% 
4.094488188976378% 
4.251968503937007% 
4.409448818897638% 
4.566929133858268% 
4.724409448818897% 
4.881889763779528% 
5.039370078740157% 
5.196850393700787% 
5.354330708661418% 
5.511811023622047% 
5.669291338582677% 
5.826771653543307% 
5.984251968503937% 
6.141732283464567% 
6.299212598425196% 
6.456692913385827% 
6.6141732283464565% 
6.771653543307086% 
6.929133858267717% 
7.086614173228346% 
7.244094488188976% 
7.4015748031496065% 
7.559055118110236% 
7.716535433070866% 

63.77952755905512% 
63.93700787401575% 
64.09448818897637% 
64.25196850393701% 
64.40944881889764% 
64.56692913385827% 
64.7244094488189% 
64.88188976377953% 
65.03937007874015% 
65.19685039370079% 
65.35433070866141% 
65.51181102362204% 
65.66929133858268% 
65.8267716535433% 
65.98425196850394% 
66.14173228346458% 
66.2992125984252% 
66.45669291338582% 
66.61417322834644% 
66.77165354330708% 
66.92913385826772% 
67.08661417322834% 
67.24409448818898% 
67.40157480314961% 
67.55905511811024% 
67.71653543307087% 
67.8740157480315% 
68.03149606299212% 
68.18897637795276% 
68.34645669291338% 
68.50393700787401% 
68.66141732283465% 
68.81889763779527% 
68.97637795275591% 
69.13385826771653% 
69.29133858267717% 
69.44881889763779% 
69.60629921259842% 
69.76377952755905% 
69.92125984251969% 
70.07874015748031% 
70.23622047244095% 
70.39370078740157% 
70.55118110236221% 
70.70866141732284% 
70.86614173228347% 
71.02362204724409% 
71.18110236220473% 
71.33858267716535% 
71.49606299212599% 
71.6

In [166]:
pred_t

,stock,pred_2020-02-25
0,916328,-2.968832
1,936365,-6.415244
2,905271,-1.854520
3,905113,-1.029102
4,905802,-5.294547
...,...,...
630,131745,-5.044087
631,69487D,-5.006380
632,68157P,-2.494472
633,9110RA,-8.192382


In [168]:
# Takes in output of pred_date_all and returns a portfolio and expected returns
def get_portfolio(predictions_df, num_stocks):
    pred_col = predictions_df.columns[-1]
    sorted_df = predictions_df.sort_values(by=pred_col, ascending=False)
    top = sorted_df.head(num_stocks)
    bot = sorted_df.tail(num_stocks)
    returns = top[pred_col].mean() - bot[pred_col].mean()
    return top, bot, returns

In [169]:
get_portfolio(pred_t, 5)

(      stock  pred_2020-02-25
 467  357366         6.750537
 334  546697         5.364659
 468  544623         4.528032
 189  921264         4.083651
 347  326996         3.460815,
       stock  pred_2020-02-25
 494  322549       -11.416555
 294  271980       -11.489440
 338  27020T       -13.082609
 295  916532       -14.547184
 491  88874X       -17.097374,
 18.36417145729065)